<a href="https://colab.research.google.com/github/majupadua/CP-Lab/blob/main/B2_image_classification_pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# **Deep Learning - Image Classification, Pré-Trained Model**
---

Seguindo o modelo apresentado em aula crie com um **Modelo Pré-Treinado** um classificador de imagens (ver as opções de imagens em *Exercício Selecionado*, abaixo). Adcionalmente ao modelo apresentado você deve incluir:

1. **Modelo e Treinamento**: parâmetros (mais que um) de *Early-Stopping*, e parâmetros e/ou camadas de *DropOut* para reduzir o overfitting.
2. **Data augumentation**: Empregue ao menos 2 funções de **pré-processamento Keras (2)** e ao menos 2 técnicas de **métodos tf.image (2)**. Outras técnicas mais avançadas são opcionais.
3. **Compare**. Compare e discuta os resultados quando comparado ao modelo anterior com o TensorFlow.





In [ ]:
#@title **Identificação do Grupo**

#@markdown Integrantes do Grupo, nome completo em orgem alfabética (*informe \<RA\>,\<nome\>*)
Aluno1 = '1115665, Adriana Fujita' #@param {type:"string"}
Aluno2 = '1115677, Daniel Henrique' #@param {type:"string"}
Aluno3 = 'None' #@param {type:"string"}
Aluno4 = 'None' #@param {type:"string"}
Aluno5 = 'None' #@param {type:"string"}



### **Escolha a opção de Exercício Selecionado**

Faça a seleção aqui: https://docs.google.com/spreadsheets/d/1Y2n0JE94CNhlQzhJudVMEvTYRE5RnuSa/edit?usp=sharing&ouid=102152623479619184236&rtpof=true&sd=true

e informe:

* **Dados**:
* **Classes**: T-shirt/top, Pullover
* **Modelo Pré Treinado**: ResNet


# **Resumo**

*Apresente um parágrafo resumindo o que é feito neste trabalho*.

# **Descrição da Solução**

*Discuta aqui os principais aspectos da solução desenvolvida (um parágrafo)*.



# **Implementação**

*Aqui o código da sua solução. Você deve explicar em linhas gerais o código.*

> ***ATENÇÃO: O código deve ser 100% executável no Colab.***



In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt

# Carregar dados e filtrar classes 0 e 2 (T-shirt/top e Pullover)
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
train_mask = np.isin(y_train, [0, 2])
test_mask = np.isin(y_test, [0, 2])
x_train, y_train = x_train[train_mask], y_train[train_mask]
x_test, y_test = x_test[test_mask], y_test[test_mask]
y_train = (y_train == 2).astype(np.int32)
y_test = (y_test == 2).astype(np.int32)

# Função preprocess: resize + grayscale to RGB + normalização
def preprocess(x, y):
    x = tf.expand_dims(x, -1)           # [28,28,1]
    x = tf.image.resize(x, [96, 96])
    x = tf.image.grayscale_to_rgb(x)    # converte para 3 canais
    x = tf.cast(x, tf.float32) / 255.0  # normaliza para [0,1]
    return x, y

# Data augmentation - 2 funções Keras
data_augmentation_keras = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
])

# Data augmentation - 2 funções tf.image
def augment_tfimage(x, y):
    x = tf.image.random_brightness(x, max_delta=0.1)
    x = tf.image.random_contrast(x, lower=0.9, upper=1.1)
    return x, y

batch_size = 16

# Dataset treino com augmentation
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = (train_ds
            .shuffle(1000)
            .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
            .map(lambda x, y: (data_augmentation_keras(x), y), num_parallel_calls=tf.data.AUTOTUNE)
            .map(augment_tfimage, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(batch_size)
            .cache()
            .prefetch(tf.data.AUTOTUNE)
           )

# Dataset teste sem augmentation
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = (test_ds
           .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
           .batch(batch_size)
           .cache()
           .prefetch(tf.data.AUTOTUNE)
          )

# Modelo com ResNet50 congelado e dropout para evitar overfitting
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.4),          # dropout mais agressivo
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# EarlyStopping com mais de um parâmetro
early_stop = callbacks.EarlyStopping(monitor='val_loss',
                                     patience=5,
                                     min_delta=0.001,
                                     restore_best_weights=True,
                                     verbose=1)

# Treinar
history = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs=30,
                    callbacks=[early_stop])
loss, acc = model.evaluate(test_ds)
print(f"\n✅ Acurácia final: {acc:.4f}")

# Plotar acurácia
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia durante o treinamento')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.grid(True)
plt.show()



Epoch 1/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 625s 816ms/step - accuracy: 0.5775 - loss: 0.6679 - val_accuracy: 0.8200 - val_loss: 0.4910
Epoch 2/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 644s 859ms/step - accuracy: 0.7336 - loss: 0.5356 - val_accuracy: 0.8280 - val_loss: 0.4011
Epoch 3/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 571s 761ms/step - accuracy: 0.7720 - loss: 0.4832 - val_accuracy: 0.8790 - val_loss: 0.3663
Epoch 4/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 638s 852ms/step - accuracy: 0.7860 - loss: 0.4599 - val_accuracy: 0.8920 - val_loss: 0.3447
Epoch 5/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 571s 761ms/step - accuracy: 0.7914 - loss: 0.4468 - val_accuracy: 0.8935 - val_loss: 0.3410
Epoch 6/30
 59/750 ━━━━━━━━━━━━━━━━━━━━ 7:37 662ms/step - accuracy: 0.8150 - loss: 0.4403

KeyboardInterrupt: 

# **Resultados**

*Discuta os resultados comparando com os do modelo anterior empregando uma rede do "zero".*



# **Referências**

Este é um item obrigatório. Inclua aqui o as referências, fontes, ou bibliografia e sites/bibliotecas empregadas em formato ABNT.

---

In [ ]:
#@title **Avaliação**
Resumo = 10 #@param {type:"slider", min:0, max:10, step:1}

Descricao_solucao = 7 #@param {type:"slider", min:0, max:10, step:1}

Implementacao = 9 #@param {type:"slider", min:0, max:10, step:1}

Resultados = 6 #@param {type:"slider", min:0, max:10, step:1}

Referencias = 7 #@param {type:"slider", min:0, max:10, step:1}

Geral = 7 #@param {type:"slider", min:0, max:10, step:1}








In [ ]:
#@title **Nota Final**
nota = Resumo + Descricao_solucao + Implementacao + Resultados + Referencias + Geral

nota = nota / 6

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
display(alunos)

Nota final do trabalho 7.7



,tia,nome,nota
0,1115665,ADRIANA FUJITA,7.7
1,1115677,DANIEL HENRIQUE,7.7
